In [238]:
# Imports
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
from datetime import datetime
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

### Momentum Screener

In [97]:
# Loading the data
ohlcv = pd.read_csv('./data/ohlcv.csv')
ohlcv["Date"] = pd.to_datetime(ohlcv['Date'], format='%Y-%m-%d')
ohlcv.set_index(["Ticker", "Date"], inplace=True)

In [118]:
# Setting dates
lastdate = pd.to_datetime('2020-12-31', format='%Y-%m-%d')
month1date = lastdate - pd.Timedelta(30,unit='d')
month3date = lastdate - pd.Timedelta(90,unit='d')
month6date = lastdate - pd.Timedelta(183,unit='d')
if(lastdate.is_leap_year):
    month12date = lastdate - pd.Timedelta(366,unit='d')
else:
    month12date = lastdate - pd.Timedelta(365,unit='d')

In [142]:
# Getting price data
metrics = []
for symbol in ohlcv.index.get_level_values(0).unique():
    try:
        last = ohlcv.loc[symbol].loc[lastdate, "Close"]
        month1 = ohlcv.loc[symbol].loc[month1date, "Close"]
        month3 = ohlcv.loc[symbol].loc[month3date, "Close"]
        month6 = ohlcv.loc[symbol].loc[month6date, "Close"]
        month12 = ohlcv.loc[symbol].loc[month12date, "Close"]
        metrics.append([symbol, last, month1, month3, month6, month12])
    except:
        print(symbol, ":complete data not available!")
metrics = pd.DataFrame(metrics, columns=["Ticker", "LastPrice", "Month1Price", "Month3Price", "Month6Price", "Month12Price"])
metrics.set_index(["Ticker"], inplace=True)
metrics

CARR :complete data not available!
LUMN :complete data not available!
OGN :complete data not available!
OTIS :complete data not available!


,LastPrice,Month1Price,Month3Price,Month6Price,Month12Price
Ticker,,,,,
MMM,170.791245,166.618942,155.347504,149.213440,166.195618
ABT,108.239662,107.251083,104.934052,89.943382,84.535339
ABBV,103.485329,100.481689,82.057129,93.323837,81.026260
ABMD,324.200012,268.209991,261.989990,255.220001,170.589996
ACN,258.786743,249.929703,219.477371,210.996979,205.320496
...,...,...,...,...,...
YUM,107.193649,105.001587,93.551544,84.566193,97.584480
ZBRA,384.329987,372.170013,258.809998,251.229996,255.440002
ZBH,153.631134,145.430283,137.058838,118.583778,148.054062


In [143]:
# Calculating percent returns
metrics["Month1Change"] = metrics["LastPrice"]/metrics["Month1Price"] - 1
metrics["Month3Change"] = metrics["LastPrice"]/metrics["Month3Price"] - 1
metrics["Month6Change"] = metrics["LastPrice"]/metrics["Month6Price"] - 1
metrics["Month12Change"] = metrics["LastPrice"]/metrics["Month12Price"] - 1

In [144]:
# Stocks sorted by 1-year change
metrics.sort_values(by="Month12Change", ascending=False)

,LastPrice,Month1Price,Month3Price,Month6Price,Month12Price,Month1Change,Month3Change,Month6Change,Month12Change
Ticker,,,,,,,,,
TSLA,705.669983,584.760010,415.089996,223.925995,83.666000,0.206769,0.700041,2.151354,7.434370
ENPH,175.470001,128.850006,90.430000,49.060001,26.129999,0.361816,0.940396,2.576641,5.715270
MRNA,104.470001,141.009995,68.809998,61.590000,19.559999,-0.259131,0.518239,0.696217,4.341002
ETSY,177.910004,154.669998,131.630005,111.209999,44.299999,0.150255,0.351592,0.599766,3.016027
PENN,86.370003,70.029999,72.750000,31.910000,25.559999,0.233329,0.187217,1.706675,2.379108
...,...,...,...,...,...,...,...,...,...
MRO,6.597597,5.796389,4.052968,5.853193,13.296273,0.138225,0.627843,0.127179,-0.503801
UAL,43.250000,45.320000,36.009998,34.450001,88.089996,-0.045675,0.201055,0.255443,-0.509025
NCLH,25.430000,22.940001,17.270000,16.420000,58.410000,0.108544,0.472496,0.548721,-0.564629


In [145]:
# Calculating percentile scores
for row in metrics.index:
    for time_period in [1,3,6,12]:
        metrics.loc[row, f'Month{time_period}ChangePercentile'] = stats.percentileofscore(metrics[f'Month{time_period}Change'], metrics.loc[row, f'Month{time_period}Change'])/100

In [162]:
# Evaluating weighted momentum score
metrics["Score"] = (4*metrics.iloc[:, 9]+3*metrics.iloc[:, 10]+2*metrics.iloc[:, 11]+1*metrics.iloc[:, 12])/4

In [163]:
# Final DataFrame
metrics

,LastPrice,Month1Price,Month3Price,Month6Price,Month12Price,Month1Change,Month3Change,Month6Change,Month12Change,Month1ChangePercentile,Month3ChangePercentile,Month6ChangePercentile,Month12ChangePercentile,Score
Ticker,,,,,,,,,,,,,,
MMM,170.791245,166.618942,155.347504,149.213440,166.195618,0.025041,0.099414,0.144610,0.027652,0.510978,0.359281,0.307385,0.391218,1.031936
ABT,108.239662,107.251083,104.934052,89.943382,84.535339,0.009217,0.031502,0.203420,0.280407,0.383234,0.197605,0.425150,0.746507,0.930639
ABBV,103.485329,100.481689,82.057129,93.323837,81.026260,0.029892,0.261138,0.108884,0.277183,0.548902,0.706587,0.245509,0.744511,1.387725
ABMD,324.200012,268.209991,261.989990,255.220001,170.589996,0.208754,0.237452,0.270277,0.900463,0.986028,0.670659,0.530938,0.966068,1.996008
ACN,258.786743,249.929703,219.477371,210.996979,205.320496,0.035438,0.179104,0.226495,0.260404,0.590818,0.540918,0.451098,0.714571,1.400699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,107.193649,105.001587,93.551544,84.566193,97.584480,0.020876,0.145824,0.267571,0.098470,0.481038,0.461078,0.528942,0.489022,1.213573
ZBRA,384.329987,372.170013,258.809998,251.229996,255.440002,0.032673,0.484989,0.529793,0.504580,0.572854,0.942116,0.884232,0.896208,1.945609
ZBH,153.631134,145.430283,137.058838,118.583778,148.054062,0.056390,0.120914,0.295549,0.037669,0.710579,0.409182,0.580838,0.407186,1.409681


In [166]:
# Ranking stocks by score
momentum_ranked = metrics.sort_values(by="Score", ascending=False)["Score"]
momentum_ranked

Ticker
ENPH    2.499501
TSLA    2.478543
MS      2.363772
FCX     2.348303
DIS     2.328343
          ...   
AMT     0.184631
COG     0.182635
PRGO    0.171657
D       0.166168
ED      0.135729
Name: Score, Length: 501, dtype: float64

### Value Screener

In [285]:
# Loading the data
info = pd.read_csv('./data/info.csv')
info.set_index(["Ticker"], inplace=True)
earnings = pd.read_csv('./data/earnings.csv')
earnings.set_index(["Ticker"], inplace=True)

In [297]:
# Calculating value-measures
metrics2 = pd.DataFrame()
metrics2["CashFlowYield"] = info["operatingCashflow"]/info["marketCap"]
metrics2["FreeCashFlowYield"] = info["freeCashflow"]/info["marketCap"]
metrics2["EarningsYield"] = earnings.groupby(by=["Ticker"]).sum().Earnings/info["marketCap"]
metrics2["RevenueYield"] = earnings.groupby(by=["Ticker"]).sum().Revenue/info["marketCap"]
metrics2["ReturnOnEquity"] = info["returnOnEquity"]
metrics2["EBITDAtoEV"] = 1.0/info["enterpriseToEbitda"]
metrics2["FreeCashFlowtoEV"] = info["freeCashflow"]/info["enterpriseValue"]
metrics2["PEGRatio"] = info["pegRatio"]
metrics2["BookValueYield"] = info["bookValue"]/info["currentPrice"]
metrics2["SalesYield"] = 1.0/info["priceToSalesTrailing12Months"]

In [298]:
# Imputing missing values
indexnames = info.index
colnames = metrics2.columns
imp = IterativeImputer(max_iter=100)
metrics2 = pd.DataFrame(imp.fit_transform(metrics2), columns=colnames, index=indexnames)

,CashFlowYield,FreeCashFlowYield,EarningsYield,RevenueYield,ReturnOnEquity,EBITDAtoEV,FreeCashFlowtoEV,PEGRatio,BookValueYield,SalesYield
Ticker,,,,,,,,,,
MMM,0.075537,0.052088,0.050382,0.290514,0.466120,0.079605,0.046699,2.20,0.127347,0.306152
ABT,0.047880,0.034146,0.028722,0.181235,0.196580,0.050520,0.032863,2.18,0.152260,0.181236
ABBV,0.096367,0.085665,0.031397,0.253175,0.488550,0.097125,0.063812,1.94,0.059231,0.253175
ABMD,0.018238,0.011062,0.009445,0.057205,0.128180,0.018400,0.011481,7.54,0.079979,0.057205
ACN,0.045319,0.037729,0.027012,0.224140,0.322100,0.038829,0.039022,3.06,0.090375,0.224140
...,...,...,...,...,...,...,...,...,...,...
YUM,0.035347,0.022213,0.029147,0.149291,0.446908,0.044629,0.019030,1.90,-0.200984,0.149291
ZBRA,0.033907,0.028587,0.020224,0.149183,0.310780,0.034009,0.032265,3.18,0.075836,0.149183
ZBH,0.031866,0.025176,0.018088,0.225844,0.046800,0.048560,0.019233,1.69,0.396685,0.225844


In [299]:
# Calculating percentile scores
for col in list(metrics2.columns):
    metrics2[f'{col}Percentile'] = metrics2[f'{col}'].apply(abs).rank(method='min', pct=True)
metrics2['PEGRatioPercentile'] = metrics2['PEGRatio'].apply(abs).rank(method='min', pct=True, ascending=False)

In [302]:
# Evaluating mean score
metrics2["Score"] = metrics2.iloc[:, 10:].mean(axis=1)

In [306]:
# Final DataFrame
metrics2

,CashFlowYield,FreeCashFlowYield,EarningsYield,RevenueYield,ReturnOnEquity,EBITDAtoEV,FreeCashFlowtoEV,PEGRatio,BookValueYield,SalesYield,...,FreeCashFlowYieldPercentile,EarningsYieldPercentile,RevenueYieldPercentile,ReturnOnEquityPercentile,EBITDAtoEVPercentile,FreeCashFlowtoEVPercentile,PEGRatioPercentile,BookValueYieldPercentile,SalesYieldPercentile,Score
Ticker,,,,,,,,,,,,,,,,,,,,,
MMM,0.075537,0.052088,0.050382,0.290514,0.466120,0.079605,0.046699,2.20,0.127347,0.306152,...,0.667327,0.681188,0.518812,0.865347,0.712871,0.710891,0.409901,0.314851,0.546535,0.607723
ABT,0.047880,0.034146,0.028722,0.181235,0.196580,0.050520,0.032863,2.18,0.152260,0.181236,...,0.435644,0.376238,0.342574,0.526733,0.407921,0.520792,0.413861,0.366337,0.342574,0.413465
ABBV,0.096367,0.085665,0.031397,0.253175,0.488550,0.097125,0.063812,1.94,0.059231,0.253175,...,0.833663,0.421782,0.453465,0.877228,0.825743,0.833663,0.471287,0.152475,0.455446,0.607723
ABMD,0.018238,0.011062,0.009445,0.057205,0.128180,0.018400,0.011481,7.54,0.079979,0.057205,...,0.063366,0.065347,0.061386,0.350495,0.065347,0.095050,0.065347,0.207921,0.061386,0.109505
ACN,0.045319,0.037729,0.027012,0.224140,0.322100,0.038829,0.039022,3.06,0.090375,0.224140,...,0.487129,0.346535,0.407921,0.732673,0.249505,0.609901,0.261386,0.235644,0.413861,0.411881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,0.035347,0.022213,0.029147,0.149291,0.446908,0.044629,0.019030,1.90,-0.200984,0.149291,...,0.251485,0.386139,0.271287,0.825743,0.344554,0.225743,0.483168,0.467327,0.271287,0.376634
ZBRA,0.033907,0.028587,0.020224,0.149183,0.310780,0.034009,0.032265,3.18,0.075836,0.149183,...,0.348515,0.231683,0.269307,0.718812,0.186139,0.508911,0.245545,0.198020,0.269307,0.319406
ZBH,0.031866,0.025176,0.018088,0.225844,0.046800,0.048560,0.019233,1.69,0.396685,0.225844,...,0.299010,0.194059,0.409901,0.097030,0.386139,0.231683,0.542574,0.712871,0.415842,0.347129


In [304]:
# Ranking stocks by score
value_ranked = metrics2.sort_values(by="Score", ascending=False)["Score"]
value_ranked

Ticker
AAL     0.954059
UAL     0.916238
DXC     0.903960
APA     0.894455
GM      0.886139
          ...   
PAYC    0.097822
ISRG    0.096832
TECH    0.095248
NOW     0.074059
TSLA    0.072079
Name: Score, Length: 505, dtype: float64